In [ ]:
import sys
# !{sys.executable} -m pip install peft wandb
# !{sys.executable} -m pip install pandas>=2.0.0
import os

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # for 1B
import gc
gc.collect()
torch.cuda.empty_cache()

Using device: cuda


In [ ]:
import torch
import transformers
from transformers import MllamaForConditionalGeneration, AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig,LlamaTokenizer
from datasets import load_dataset
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datetime import datetime

if 'COLAB_GPU' in os.environ:
    from google.colab import output
    output.enable_custom_widget_manager()

In [4]:
# base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# base_model = "meta-llama/Llama-3.2-90B-Vision-Instruct"
# base_model = "meta-llama/Llama-3.2-11B-Vision"
base_model = "meta-llama/Llama-3.2-11B-Vision-Instruct"
# base_model = "meta-llama/Llama-3.2-1B"
# base_model = "meta-llama/Llama-3.2-3B"
# base_model = "meta-llama/Meta-Llama-3.1-70B"
# base_model = "meta-llama/Meta-Llama-3.1-405B"
new_model = "llama-32-11B_chat-building"
import torch
torch_dtype = torch.float16
attn_implementation = "eager"
# !{sys.executable} -m pip install bitsandbytes
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)


In [6]:
from transformers import MllamaForConditionalGeneration, AutoProcessor
# 11B 
model = MllamaForConditionalGeneration.from_pretrained(
    base_model,
    quantization_config=bnb_config, # BitsAndBytesConfig(load_in_8bit=True), #bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
)
    

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
import json
#  with open('/srv/data1/fuxiaoqin/Downloads/EngineeringReference1_10_QAGPT4_10P.json', 'r', encoding='utf-8') as file: 
#  with open('/srv/data1/fuxiaoqin/EnergyPlus/engineering-reference_iqa_testGPT4omini.json', 'r', encoding='utf-8') as file:  
# with open('/srv/data1/fuxiaoqin/Downloads/EngineeringReference1_6_TESTQAGPT4omini.json', 'r', encoding='utf-8') as file:  
# with open('/home/fuxiaoqin/Downloads/EngineeringReference1_All_TESTQAGPT4omini.json', 'r', encoding='utf-8') as file:  
with open('/srv/data1/fuxiaoqin/EnergyPlus/EI_iqa_testGPT4omini.json', 'r', encoding='utf-8') as file:  
    json_obj = json.load(file)  

# print(json_obj)
# print(json_obj[0])
# data_chunkis = []
data_instructions = []
data_inputs = []
data_outputs = []
for piece in json_obj:
    # print("piece", piece['input'])
    # data_chunkis.append(piece['chunki'])
    data_instructions.append(piece['instruction'])
    data_inputs.append(piece['input'])
    data_outputs.append(piece['output'])





In [8]:
print(len(data_instructions))
print(len(data_inputs))
print(len(data_outputs))
# print(data_instructions)

261
261
261


In [9]:
#!{sys.executable} -m pip uninstall -q wandb
import os
os.environ["WANDB_DISABLED"] = "true"

In [10]:
# from accelerate import PartialState
# device_map={"": PartialState().process_index}
# # device_index = Accelerator().process_index
# # device_map = {"": device_index}
# # Load model
# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     quantization_config=bnb_config, # BitsAndBytesConfig(load_in_8bit=True), #bnb_config,
#     device_map=device_map,
#     attn_implementation=attn_implementation,
# )

from trl import SFTTrainer, setup_chat_format
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
# model, tokenizer = setup_chat_format(model, tokenizer)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [11]:
def save_results_csv(fileName, instructions, inputs, outputs, predictions):
    min_len = len(instructions) # 296 # 11 # 129 # 156 #2961 # 1297 #129 #296 # 359 # min (len(instructions), len(inputs), len(outputs), len(predictions))
    wf = open(fileName, 'w', encoding="utf8")
    wf.write("Instruction,Input,Output,Predicted\n")
    for i in range(min_len):
        wf.write(str(instructions[i])+','+str(inputs[i])+',"'+str(outputs[i])+'","'+str(predictions[i])+'"\n')
    wf.close()

In [12]:
references = data_outputs
print("len(references)", len(references))
# print("len(pretrained_responses)", len(pretrained_responses))

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from numpy import *
from rouge import Rouge
from bert_score import score

def compute_bleu(reference, hypothesis):
    reference = [reference.split()]
    hypothesis = hypothesis.split()
    smoothie = SmoothingFunction().method4
    return sentence_bleu(reference, hypothesis, smoothing_function=smoothie)

def save_metrics_csv(fileName, trainTime, bleus, rouge1, rouge2, rouge3, b_r, b_p, b_f):
    bleu = mean(bleus)
    r1_r=rouge1['r']
    r1_p=rouge1['p']
    r1_f=rouge1['f']
    r2_r=rouge2['r']
    r2_p=rouge2['p']
    r2_f=rouge2['f']
    r3_r=rouge3['r']
    r3_p=rouge3['p']
    r3_f=rouge3['f']    
    b_r = str(b_r).replace("tensor(","").replace(")","")
    b_p = str(b_p).replace("tensor(","").replace(")","")
    b_f = str(b_f).replace("tensor(","").replace(")","")
    if os.path.exists(fileName):
        r = open(fileName, 'a', encoding="utf8")
        r.write(str(trainTime)+","+str(bleu)+","+str(r1_r)+","+str(r1_p)+","+str(r1_f)+","+str(r2_r)+","+str(r2_p)+","+str(r2_f)+","+str(r3_r)+","+str(r3_p)+","+str(r3_f)+","+str(b_r)+","+str(b_p)+","+str(b_f)+",\n")
        r.close()
    else:
        r = open(fileName, 'w', encoding="utf8")
        r.write("Training,BLEU,ROUGE-1,,,ROUGE-2,,,ROUGE-L,,,BERT,,,Fineturning\n")
        r.write("Time,,r ,p,f,r ,p,f,r ,p,f,R,P,F1,Time\n")
        r.write(str(trainTime)+","+str(bleu)+","+str(r1_r)+","+str(r1_p)+","+str(r1_f)+","+str(r2_r)+","+str(r2_p)+","+str(r2_f)+","+str(r3_r)+","+str(r3_p)+","+str(r3_f)+","+str(b_r)+","+str(b_p)+","+str(b_f)+",\n")
        r.close()  
        
def compute_save_metrics(references, responses, fileName, trainingTime):
    # BLEU scores
    bleu_scores = [compute_bleu(ref, hypo) for ref, hypo in zip(references, responses)]
    print("BLEU scores with fineturning:", mean(bleu_scores))
    
    # ROUGE scores
    from rouge import Rouge
    rouge = Rouge()
    rouge1_r=0
    rouge1_p=0
    rouge1_f=0
    rouge2_r=0
    rouge2_p=0
    rouge2_f=0
    rougel_r=0
    rougel_p=0
    rougel_f=0
    rouge1 = {'r':0, 'p':0, 'f':0}
    rouge2 = {'r':0, 'p':0, 'f':0}
    rougel = {'r':0, 'p':0, 'f':0}
    try:
        rouge_scoress = rouge.get_scores(responses, references)
        for rouge_scores in rouge_scoress:
            rouge1=rouge_scores['rouge-1']
            rouge1_r += rouge1['r']
            rouge1_p += rouge1['p']
            rouge1_f += rouge1['f']
            rouge2=rouge_scores['rouge-2']   
            rouge2_r += rouge2['r']
            rouge2_p += rouge2['p']
            rouge2_f += rouge2['f']
            rougel=rouge_scores['rouge-l']       
            rougel_r += rougel['r']
            rougel_p += rougel['p']
            rougel_f += rougel['f']
        lenRouge = len(rouge_scoress)
        rouge1_r /= lenRouge
        rouge1_p /= lenRouge
        rouge1_f /= lenRouge
        rouge2_r /= lenRouge
        rouge2_p /= lenRouge
        rouge2_f /= lenRouge
        rougel_r /= lenRouge
        rougel_p /= lenRouge
        rougel_f /= lenRouge        
    except:
        print("No rouge scores")
    # ROUGE-1 Scores
    print(f"ROUGE-1 scores r, p, f with fine-turning: {rouge1_r}, {rouge1_p}, {rouge1_f} \n")    
    # ROUGE-2 scores
    print(f"ROUGE-2 scores r, p, f withfine-turning: {rouge2_r}, {rouge2_p}, {rouge2_f} \n")    
    # ROUGE-L scores
    print(f"ROUGE-l scores r, p, f with fine-turning: {rougel_r}, {rougel_p}, {rougel_f} \n")
    # ROUGE-W scores    
    
    #  BERTScore
    from bert_score import score
    # Calculate BERTScore
    P, R, F1 = score(responses, references, lang="en", rescale_with_baseline=True, verbose=True)
    print(f"type(P): {type(P)}\n")
    P = torch.mean(P)
    R = torch.mean(R)
    F1 = torch.mean(F1)
    # Print Rbert, Pbert and Fbert
    
    print(f"BERTScore-Precision with fineturning: {P}\n")
    print(f"\nBERTScore-Recall with fineturning: {R}\n")
    print(f"BERTScore-F-Measure with fineturning: {F1}")

    save_metrics_csv(fileName, trainingTime, bleu_scores, rouge1, rouge2, rougel, P, R, F1)

len(references) 261


In [13]:
def get_responses(model, data_inputs):
    responses = []
    for dp in data_inputs:
        inputs = tokenizer(dp, return_tensors="pt").to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=64)
        response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        text = response_text.replace(dp,"").replace("#"," ").replace("\n"," ")
        # text = response_text.replace(dp,"").replace("#"," ").replace("\n"," ").split(", ")[0].split(". ")[0].strip()+"."
        # if len(text)<3:
        #     text = response_text.replace(dp,"").replace("#"," ").replace("\n"," ")
        print(f"text={text}")
        response = text
        responses.append(response)
    print(f"responses = {responses}")
    print("len(responses) =", len(responses))
    return responses

In [14]:
# pretrained_responses = get_responses(model, data_inputs)
# # # save_results_csv("/srv/data1/fuxiaoqin/Results_10_LM32_1B_10P.csv", data_instructions, data_inputs, data_outputs, pretrained_responses)
# # # compute_save_metrics(references, pretrained_responses, "/srv/data1/fuxiaoqin/Metrics_10_LM32_1B_10P.csv","LM32_1B")

# save_results_csv("/srv/data1/fuxiaoqin/Results_E_27_11B_before.csv", data_instructions, data_inputs, data_outputs, pretrained_responses)
# compute_save_metrics(references, pretrained_responses, "/srv/data1/fuxiaoqin/Metrics_E_27_11B_before.csv","LM32_11B")


In [15]:
# sys.exit()

In [16]:
import os
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from peft.optimizers import create_loraplus_optimizer
from transformers import Trainer
import bitsandbytes as bnb
# from peft.optimizers import create_lorafa_optimizer
from transformers import Trainer, get_cosine_schedule_with_warmup

from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [17]:
    
def formatting_func(example):
    text = f"### Question: {example['input']}\n ### Answer: {example['output']}"+tokenizer.eos_token
    return text
        
def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt))

def formatting_func(example):
    text = f"### Question: {example['input']}\n ### Answer: {example['output']}"+tokenizer.eos_token
    return text
        
def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt))

In [18]:
def generate_and_tokenize_prompt2(prompt, max_length):
    result = tokenizer(
        formatting_func(prompt),
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [19]:
from peft import LoraConfig, EvaConfig
peft_config = LoraConfig(
    # init_lora_weights = "eva",
    # eva_config = EvaConfig(rho = 2.0),

    # use_dora=True,
    r = 1,  #16, #128,  #32, #16, # 1,  #16
    lora_alpha = 1,  #32, #64, #32, # 1, #32,
    lora_dropout=0.05,

    # use_rslora = True,

        
    bias= "none", # "all", # "none",
    # task_type="QUESTION_ANS",     # for 1B and 3B
    task_type="CAUSAL_LM",   # for 11B
    # target_modules=['k_proj', 'q_proj', 'v_proj'], # for 11B
    # target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj', 'fc1', 'fc2'], # for 11B
    target_modules=['q_proj'], #  'fc1', 'fc2'], # for 11B
    # target_modules="all-linear", # for 1B and 3B
    # modules_to_save=["embed_tokens", "lm_head"]
        # target_modules=['q_proj']
)

In [ ]:
from peft import LoraConfig, get_peft_model
from peft.optimizers import create_loraplus_optimizer

def QA_training(model, QA_folder, QA_file, epochs=1):
    data_dict = {
        # "train": os.path.join("/home/fuxiaoqin/Downloads/", "A311.json")
        # "train": os.path.join("/mnt/data1/fuxiaoqin/Downloads/", "Ashrae_2.json")
        "train": os.path.join(QA_folder, QA_file)
        # "train": os.path.join("/home/fuxiaoqin/Downloads/", "EngineeringReference_QAGPT4.json")
    }
  
                
    dataset = load_dataset("json", data_files=data_dict, split = "train")

    dataset = dataset.map(generate_and_tokenize_prompt)
    

    #     return tokenizer(formatting_func(prompt))
    dataset = dataset.map(generate_and_tokenize_prompt)
    

    
    max_length =512 
    
    
    dataset = dataset.map(generate_and_tokenize_prompt2, max_length)
    # dataset['text'][3]
    
    # dataset = dataset.train_test_split(test_size=0.1)
    print(dataset)
    

    #     # target_modules=['q_proj']
    # )
    model = get_peft_model(model, peft_config)   
    # model = torch.nn.DataParallel(model, device_ids=[0,1,2])

    optimizer = create_loraplus_optimizer(
        model=model,
        optimizer_cls=bnb.optim.Adam8bit,
        lr=5e-5,
        loraplus_lr_ratio=16,
    )
    scheduler = None

    
    training_arguments = TrainingArguments(
        output_dir=new_model,
        per_device_train_batch_size=1,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=2,
        # optim="paged_adamw_32bit",
        num_train_epochs=1,
        # eval_strategy="steps",
        # eval_steps=0.2,
        logging_steps=10,
        warmup_steps=10,
        logging_strategy="steps",
        learning_rate=1e-5,
        fp16=False, # True, #False,
        bf16=True, # False, #True,
        group_by_length=True,
        # report_to="wandb"
    )

    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        # eval_dataset=dataset["test"],
        peft_config=peft_config,
        # max_seq_length=512,
        # dataset_text_field="text",
        tokenizer=tokenizer,
        args=training_arguments,
        optimizers=(optimizer, scheduler),
        # use_dora=True,
        # packing= False,
    )

    print(torch.cuda.is_available())
    for i in range(epochs):
        trainer.train()
        
    return model

In [ ]:
import time
start_time = time.time()

model = QA_training(model, "/srv/data1/fuxiaoqin/inputs/", "engineering-reference_iqa.json",1)

end_time = time.time()

lora_time = end_time - start_time
print("Fine-tuning: {:.2f}seconds".format(lora_time))

In [ ]:

print("Fine-tuning: {:.2f}seconds".format(lora_time))

In [ ]:
responses = get_responses(model, data_inputs)

In [ ]:
len(responses)

In [ ]:
# save_results_csv("/srv/data1/fuxiaoqin/Results_10_LM32_1B_10P_QA.csv", data_instructions, data_inputs, data_outputs, responses)
# compute_save_metrics(references, responses, "/srv/data1/fuxiaoqin/Metrics_10_LM32_1B_10P_QA.csv","LM32_1B")
save_results_csv("/srv/data1/fuxiaoqin/Results_E_27_11B.csv", data_instructions, data_inputs, data_outputs, responses)
compute_save_metrics(references, responses, "/srv/data1/fuxiaoqin/Metrics_E_27_11B.csv",str(lora_time))